In [1]:
import csv
import requests

In [2]:

class FuelStationDataProcessor:
    def __init__(self, api_key):
        self.url = 'https://community-nrel-national-renewable-energy-laboratory.p.rapidapi.com/api/alt-fuel-stations/v1/nearest.json'
        self.location = '433 Bryant St., San Francisco'
        self.fuel_type = 'ELEC'
        self.limit = 50
        self.offset = 0
        self.api_key = api_key
        self.headers = {
            'X-RapidAPI-Key': self.api_key,
            'X-RapidAPI-Host': 'community-nrel-national-renewable-energy-laboratory.p.rapidapi.com'
        }
        self.public_data = []
        self.private_data = []
        self.others_data = []

    def fetch_data(self):
        while True:
            querystring = {
                'location': self.location,
                'fuel_type': self.fuel_type,
                'limit': self.limit,
                'offset': self.offset
            }
            response = requests.get(self.url, headers=self.headers, params=querystring)

            if response.status_code == 200:
                data = response.json()['fuel_stations']
                if not data:
                    break

                for station in data:
                    group = station.get('groups_with_access_code', '').lower()
                    if group == 'public':
                        self.public_data.append(station)
                    elif group == 'private':
                        self.private_data.append(station)
                    else:
                        self.others_data.append(station)

                self.offset += self.limit
            else:
                print(f'Failed to get data for offset {self.offset}. Skipping...')
                break

    def save_data_to_csv(self, data, filename):
        if data:
            keys = list(data[0].keys()) + ['federal_agency', 'ev_network_ids']
            with open(filename, mode='w', newline='') as csv_file:
                writer = csv.DictWriter(csv_file, fieldnames=keys)
                writer.writeheader()
                writer.writerows(data)
            print(f'Data has been saved to {filename}.')
        else:
            print('No data to save.')

    def process_data(self):
        self.fetch_data()
        self.save_data_to_csv(self.public_data, 'public_stations.csv')
        self.save_data_to_csv(self.private_data, 'private_stations.csv')
        self.save_data_to_csv(self.others_data, 'others_stations.csv')

if __name__ == '__main__':
    api_key = 'YOUR_RAPIDAPI_KEY_HERE'
    db = FuelStationDataProcessor(api_key='51b07a06c6mshbaae7fd65806f5cp13da34jsn59c5aa35baa7')
    db.process_data()


Data has been saved to public_stations.csv.
Data has been saved to private_stations.csv.
Data has been saved to others_stations.csv.
